In [1]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"


driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [21]:
source = "df5449c3-bc39-4a3f-9d0f-8a0ba90d1f7c"
sentence = "Monitor your stock in real-time, manage your inventory across all locations"

records, summary, key  = driver.execute_query("""
    MATCH (req:Requirement) WHERE req.id = $id
    WITH req
    MATCH (target:Requirement) WHERE target.id <> $id
    WITH target, vector.similarity.cosine(target.`fastrp-embedding`, req.`fastrp-embedding`) as score
    RETURN target.id as id, target.app as app, target.sentence as sentence, score ORDER BY score DESC LIMIT 5""", id=source)

result = []
for record in records:
    result.append({"app": record['app'], "sentence": record['sentence'], "score": record['score']})

In [22]:
import pandas as pd

df_result = pd.DataFrame(result)

In [23]:
df_result[:5]

,app,sentence,score
0,Odoo Inventory,Monitor how long items have been in the wareho...,0.725182
1,Odoo Sales,Configure Odoo Sales to automatically display ...,0.686429
2,SMS,The system shall provide the functionality to ...,0.644496
3,SIMS,The system must provide a report that displays...,0.636702
4,Odoo Purchase,"Keep track of stock, incoming orders and produ...",0.615479


In [33]:
subject1 = "system"
predicate1 = "monitor"
object1 = "stock"

subject2 = "system"
predicate2 = "manage"
object2 = "inventory"

uid = "a2608049-76a7-40c4-8ad6-6b922dbe2cc6"

In [37]:
from sentence_transformers import SentenceTransformer

def sentence_transformer_embedding(text):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(text)
    return embeddings

In [34]:
cql_query = "MERGE (req:Requirement {id: '" + uid +"', app: 'Demo'}) \n"
cql_query += f"MERGE (s1:CORE" + " {name: '" + subject1 + "'}) \n"
cql_query += f"MERGE (p1:VERB" + " {name: '" + predicate1 + "'}) \n"
cql_query += f"MERGE (o1:CORE" + " {name: '" + object1 + "'}) \n"

cql_query += f"MERGE (s2:CORE" + " {name: '" + subject2 + "'}) \n"
cql_query += f"MERGE (p2:VERB" + " {name: '" + predicate2 + "'}) \n"
cql_query += f"MERGE (o2:CORE" + " {name: '" + object2 + "'}) \n"

cql_query += f"MERGE (s1)-[:RELATED_TO]->(p1)-[:RELATED_TO]->(o1) \n"
cql_query += f"MERGE (s2)-[:RELATED_TO]->(p2)-[:RELATED_TO]->(o2) \n"

cql_query += f"MERGE (req)-[:CONTAINS]->(s1) \n"
cql_query += f"MERGE (req)-[:CONTAINS]->(p1) \n"
cql_query += f"MERGE (req)-[:CONTAINS]->(o1) \n"

cql_query += f"MERGE (req)-[:CONTAINS]->(s2) \n"
cql_query += f"MERGE (req)-[:CONTAINS]->(p2) \n"
cql_query += f"MERGE (req)-[:CONTAINS]->(o2) \n"

In [35]:
session.run(cql_query)

In [39]:
records, summary, key = driver.execute_query("MATCH(n:CORE|USER|PLATFORM|HARDWARE|VERB) WHERE n.id = $id return n.name", id=uid)
for record in records:
    embedding = sentence_transformer_embedding(record['n.name'])
    session.run("MATCH (n {name: $name}) WHERE n.embedding is null SET n.embedding = $embedding", name=record['e.name'], embedding=embedding)

In [47]:
session.run("""
MATCH (source:Requirement)-[:CONTAINS]->(target:CORE|VERB|USER|PLATFORM|HARDWARE)
RETURN gds.graph.project(
  'myGraph7',
  source,
  target,
  {
    sourceNodeProperties: {},
    targetNodeProperties: target { .embedding }
  },
  { undirectedRelationshipTypes: ['*'] }
)
""")

In [48]:
session.run("""
CALL gds.fastRP.write('myGraph7',
  {
    embeddingDimension: 768,
    writeProperty: 'graph-fastrp-embedding'
  }
)
YIELD nodePropertiesWritten
""")

In [49]:
records, summary, key  = driver.execute_query("""
    MATCH (req:Requirement) WHERE req.id = $id
    WITH req
    MATCH (target:Requirement) WHERE target.id <> $id
    WITH target, vector.similarity.cosine(target.`graph-fastrp-embedding`, req.`graph-fastrp-embedding`) as score
    RETURN target.id as id, target.app as app, target.sentence as sentence, score ORDER BY score DESC LIMIT 5""", id=uid)

result = []
for record in records:
    result.append({"app": record['app'], "sentence": record['sentence'], "score": record['score']})

In [50]:
df_result = pd.DataFrame(result)
df_result[:5]

,app,sentence,score
0,SMS,The system shall update the stock to reflect t...,0.667610
1,SMS,The system shall enable the Sales Assistant to...,0.661088
2,SCATS,The system must allow the Sales Manager to rev...,0.656658
3,SCATS,"If the ""Total Largest Revenue"" option is selec...",0.652254
4,SCATS,"After the sale is completed, the system must u...",0.652246
